In [29]:
import kagglegym
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from timeit import timeit


In [39]:
env = kagglegym.make()
o = env.reset()
#o.train = o.train[:1000]
excl = [env.ID_COL_NAME, env.SAMPLE_COL_NAME, env.TARGET_COL_NAME, env.TIME_COL_NAME]
col = [c for c in o.train.columns if c not in excl]



In [40]:
train = pd.read_hdf('../input/train.h5')
train = train[col]
d_mean= train.median(axis=0)



In [41]:
train = o.train[col]
n = train.isnull().sum(axis=1)
for c in train.columns:
    train[c + '_nan_'] = pd.isnull(train[c])
    d_mean[c + '_nan_'] = 0
train = train.fillna(d_mean)
train['znull'] = n
n = []



/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [42]:
rfr = ExtraTreesRegressor(n_estimators=100, max_depth=4, n_jobs=-1, random_state=17, verbose=0)
model1 = rfr.fit(train, o.train['y'])
train = []

#https://www.kaggle.com/bguberfain/two-sigma-financial-modeling/univariate-model-with-clip/run/482189
low_y_cut = -0.075
high_y_cut = 0.075

start=timeit.timeit()

y_is_above_cut = (o.train.y > high_y_cut)
y_is_below_cut = (o.train.y < low_y_cut)
y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)
model2 = LinearRegression(n_jobs=-1)
model2.fit(np.array(o.train[col].fillna(d_mean).loc[y_is_within_cut, 'technical_20'].values).reshape(-1,1), o.train.loc[y_is_within_cut, 'y'])

#https://www.kaggle.com/ymcdull/two-sigma-financial-modeling/ridge-lb-0-0100659
ymean_dict = dict(o.train.groupby(["id"])["y"].median())

end = timeit.timeit()
print(end - start)


-0.006740911994711496


In [43]:
start=timeit.timeit()

while True:
    test = o.features[col]
    n = test.isnull().sum(axis=1)
    for c in test.columns:
        test[c + '_nan_'] = pd.isnull(test[c])
    test = test.fillna(d_mean)
    test['znull'] = n
    pred = o.target
    test2 = np.array\
    (o.features[col].fillna(d_mean)['technical_20'].values).reshape(-1,1)
    pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * 0.65)+\
    (model2.predict(test2).clip(low_y_cut, high_y_cut) * 0.35)
    pred['y'] = pred.apply(lambda r: 0.95 * r['y'] + \
                           0.05 * ymean_dict[r['id']] \
                           if r['id'] in ymean_dict else r['y'], axis = 1)
    pred['y'] = [float(format(x, '.6f')) for x in pred['y']]
    o, reward, done, info = env.step(pred)
    if done:
        print("el fin ...", info["public_score"])
        break
    if o.features.timestamp[0] % 100 == 0:
        print(reward)
        
end = timeit.timeit()
print(end - start)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


-0.317732370655
-0.182276665545


KeyboardInterrupt: 

In [50]:
def fitInitialScript(low_y_cut=-0.075, high_y_cut=0.075, coeff1=0.65,\
                     coeff2=0.95):
    env = kagglegym.make()
    o = env.reset()
    
    y_is_above_cut = (o.train.y > high_y_cut)
    y_is_below_cut = (o.train.y < low_y_cut)
    y_is_within_cut = (~y_is_above_cut & ~y_is_below_cut)
    model2 = LinearRegression(n_jobs=-1)
    model2.fit(np.array(o.train[col].fillna(d_mean).loc[y_is_within_cut, 'technical_20'].values).reshape(-1,1), o.train.loc[y_is_within_cut, 'y'])
    #https://www.kaggle.com/ymcdull/two-sigma-financial-modeling/ridge-lb-0-0100659
    ymean_dict = dict(o.train.groupby(["id"])["y"].median())
    
    while True:
        test = o.features[col]
        n = test.isnull().sum(axis=1)
        for c in test.columns:
            test[c + '_nan_'] = pd.isnull(test[c])
        test = test.fillna(d_mean)
        test['znull'] = n
        pred = o.target
        test2 = np.array\
        (o.features[col].fillna(d_mean)['technical_20'].values).reshape(-1,1)
        pred['y'] = (model1.predict(test).clip(low_y_cut, high_y_cut) * coeff1)+\
        (model2.predict(test2).clip(low_y_cut, high_y_cut) * (1-coeff1))
        pred['y'] = pred.apply(lambda r: coeff2 * r['y'] + \
                           (1-coeff2) * ymean_dict[r['id']] \
                           if r['id'] in ymean_dict else r['y'], axis = 1)
        pred['y'] = [float(format(x, '.6f')) for x in pred['y']]
        o, reward, done, info = env.step(pred)
        if done:
            print("low=",low_y_cut, "high=", high_y_cut, "coeff1=",\
                  coeff1, "coeff2=", coeff2, \
                  "r=",info["public_score"])
            break
#         if o.features.timestamp[0] % 100 == 0:
#             print(reward)

In [47]:
fitInitialScript(-0.086093,0.093497)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


el fin ... 0.0212274000456


In [49]:
n=2
x=-.075+np.arange(-n,n+1)/(n)*.01
y=.075+np.arange(-n,n+1)/(n)*.01
for i in [(a,b) for a in x for b in y]:
    fitInitialScript(*i)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


low= -0.085 high= 0.065 r= 0.0209034018658
low= -0.085 high= 0.07 r= 0.0213959546743
low= -0.085 high= 0.075 r= 0.0216419710708
low= -0.085 high= 0.08 r= 0.0216557887025
low= -0.085 high= 0.085 r= 0.0215369685407
low= -0.08 high= 0.065 r= 0.0212120087397
low= -0.08 high= 0.07 r= 0.0215379611079
low= -0.08 high= 0.075 r= 0.0216506500503
low= -0.08 high= 0.08 r= 0.0215438737015
low= -0.08 high= 0.085 r= 0.0213106869652
low= -0.075 high= 0.065 r= 0.0214506190502
low= -0.075 high= 0.07 r= 0.0215877595609
low= -0.075 high= 0.075 r= 0.0215442626819
low= -0.075 high= 0.08 r= 0.0212951988375
low= -0.075 high= 0.085 r= 0.020927671549
low= -0.07 high= 0.065 r= 0.021580447962
low= -0.07 high= 0.07 r= 0.0215096098464
low= -0.07 high= 0.075 r= 0.0212877028251
low= -0.07 high= 0.08 r= 0.0208742867317
low= -0.07 high= 0.085 r= 0.0203466129444
low= -0.065 high= 0.065 r= 0.0215203751963
low= -0.065 high= 0.07 r= 0.02120781505
low= -0.065 high= 0.075 r= 0.0207703824355
low= -0.065 high= 0.08 r= 0.020152

In [ ]:
n=2
x=-.075+np.arange(-n,n+1)/(n)*.01
y=.075+np.arange(-n,n+1)/(n)*.01
w=.65+np.arange(-n,n+1)/(n)*.05
z=.95+np.arange(-n,n+1)/(n)*.05
for i in [(c,d) for c in w for d in z]:
    fitInitialScript(-.08,.075,*i)

/Users/jingguo/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
